In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from itertools import combinations
from sklearn.model_selection import KFold

In [2]:
def calculate_aic(y, x):
    model = sm.OLS(y, sm.add_constant(x)).fit()
    return model.aic

In [3]:
def perform_k_fold_cross_validation(data, target_variable, selected_variables, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    aic_values = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index]
        test_data = data.iloc[test_index]

        aic = calculate_aic(train_data[target_variable], train_data[selected_variables])
        aic_values.append(aic)

    return np.mean(aic_values)

In [4]:
def stepwise_regression(data, target_variable, max_features=None, k=5):
    independent_variables = data.columns.drop(target_variable)
    best_aic = np.inf
    best_model = None
    selected_variables = []

    if max_features is None:
        max_features = len(independent_variables)

    while True:
        remaining_variables = list(set(independent_variables) - set(selected_variables))
        aic_candidates = []

        for feature in remaining_variables:
            selected_vars = selected_variables + [feature]
            if data[selected_vars].isnull().values.any(): # check if the selected variables contain any NaNs
                continue
            if np.isinf(data[selected_vars].values).any(): # check if the selected variables contain any infinite values
                continue
            
            aic = perform_k_fold_cross_validation(data, target_variable, selected_vars, k)
            aic_candidates.append((feature, aic))

        if not aic_candidates:
            break

        aic_candidates.sort(key=lambda x: x[1])
        best_candidate = aic_candidates[0]

        if best_candidate[1] < best_aic:
            selected_variables.append(best_candidate[0])
            best_aic = best_candidate[1]
            best_model = sm.OLS(data[target_variable], sm.add_constant(data[selected_variables])).fit()

            if len(selected_variables) == max_features:
                break
        else:
            break

    return best_model, selected_variables

In [5]:
# Example usage:
data = pd.read_csv('D:/Mid_review_presentation/Thesis_writing/Chapters/Chapter_4/Khadakwasala/Training_Input/Chlorophyll.csv')  # Load your dataset
target_variable = 'Chlorophyll'  # Replace with your target column name
max_features = 6  # Set the maximum number of features to include in the model (optional)
k = 5  # Set the number of folds for cross-validation

In [6]:
best_model, selected_features = stepwise_regression(data, target_variable, max_features, k)
print("Selected features:", selected_features)
print(best_model.summary())

Selected features: ['B3/B2', 'B4', 'B3+B4', 'B2', 'B5', 'B2+B3']
                            OLS Regression Results                            
Dep. Variable:            Chlorophyll   R-squared:                       0.896
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     17.31
Date:                Fri, 05 May 2023   Prob (F-statistic):           0.000122
Time:                        23:54:28   Log-Likelihood:                 69.404
No. Observations:                  16   AIC:                            -126.8
Df Residuals:                      10   BIC:                            -122.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

C:\Users\Admin\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [7]:
# Write the results to a file
with open('results_chl_Bear.txt', 'w') as f:
    f.write(f"Selected features: {selected_features}\n")
    f.write(str(best_model.summary()))

C:\Users\Admin\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [8]:
pwd

'C:\\Users\\Admin'

In [9]:
# Get the coefficients and equation for the multiple linear regression model
coefficients = best_model.params
equation = "Chlorophyll = "
for i, feature in enumerate(selected_features):
    coefficient = round(coefficients[i+1], 4)
    equation += f"{coefficient}*{feature} + "
intercept = round(coefficients[0], 4)
equation += f"{intercept}"

In [10]:
# Display the coefficients and equation for the multiple linear regression model
print("Coefficients:", coefficients)
print("Equation:", equation)

Coefficients: const    0.314121
B3/B2   -0.155144
B4      -0.955064
B3+B4    1.537626
B2      -2.124382
B5      -0.492075
B2+B3    0.368308
dtype: float64
Equation: Chlorophyll = -0.1551*B3/B2 + -0.9551*B4 + 1.5376*B3+B4 + -2.1244*B2 + -0.4921*B5 + 0.3683*B2+B3 + 0.3141


In [11]:
# Make predictions using the best model
X = sm.add_constant(data[selected_features])
y_pred = best_model.predict(X)

In [12]:
# Add the predicted values to the original data
data['Predicted Chlorophyll'] = y_pred

In [13]:
# Display the original and predicted values of Turbidity
print("Original and Predicted Chlorophyll:")
print(data[[target_variable, 'Predicted Chlorophyll']])

Original and Predicted Chlorophyll:
    Chlorophyll  Predicted Chlorophyll
0          0.15               0.153012
1          0.14               0.138450
2          0.18               0.174975
3          0.15               0.147164
4          0.14               0.141465
5          0.15               0.143009
6          0.16               0.162625
7          0.15               0.151684
8          0.15               0.151070
9          0.15               0.152909
10         0.15               0.147872
11         0.15               0.153760
12         0.17               0.167454
13         0.15               0.149018
14         0.15               0.154067
15         0.16               0.161465


In [14]:
# Save the original and predicted turbidity values to an Excel file
output_data = data[[target_variable, 'Predicted Chlorophyll']]
output_file_path = 'D:/Mid_review_presentation/Thesis_writing/Chapters/Chapter_4/Khadakwasala/Training_output/Chlorophyll_Khd_4.xlsx'
output_data.to_excel(output_file_path, index=False)